In [1]:
#https://motius.breezy.hr/p/0b622aa1163e01-join-our-tech-community-as-working-student-m-f-d/apply?utm_source=pt-linkedin&utm_medium=job-posting

In [2]:
"""!pip install datasets
!pip install transformers
!pip install sentence_transformers
!pip install peft
!pip install huggingface_hub
!pip install --upgrade accelerate
!pip install --upgrade bitsandbytes
!pip install --upgrade torchmetrics
!pip install --upgrade deepspeed
"""

'!pip install datasets\n!pip install transformers\n!pip install sentence_transformers\n!pip install peft\n!pip install huggingface_hub\n!pip install --upgrade accelerate\n!pip install --upgrade bitsandbytes\n!pip install --upgrade torchmetrics\n!pip install --upgrade deepspeed\n'

In [13]:
from transformers import  AutoModel,AutoTokenizer,AutoModelForMaskedLM , Trainer,TrainingArguments,\
BitsAndBytesConfig,pipeline,default_data_collator,DataCollatorWithPadding,DataCollatorForLanguageModeling
from transformers.utils import move_cache
#from chat_template_utils import get_json_schema
#from llama_cpp import Llama
from utils import *
from peft import *
import datasets
#import torchmetrics
import torch
from torch.utils.data.dataloader import DataLoader,Dataset
#from hqq.engine.hf import HQQModelForCausalLM
#from hqq.models.hf.base import AutoHQQHFModel
from huggingface_hub import snapshot_download
#import deepspeed
import os
import pandas as pd
import json
#from var_dump import var_dump
from bs4 import BeautifulSoup,PageElement,Comment
import matplotlib.pyplot as plt
import seaborn as sns

cache_dir='/var/tmp/.cache/' #'/proj/ciptmp/ix05ogym/.cache/'
output_dir = cache_dir+'outputs/'

move_cache(cache_dir)

def pd_full_screen():
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    
def pd_normal_screen():
    pd.set_option('display.max_rows', 10)
    pd.set_option('display.max_columns', 5)


0it [00:00, ?it/s]

In [60]:
train_dataset = pd.read_excel('train_dataset.xlsx',index_col=0)
#train_dataset = train_dataset.drop_duplicates()
#train_dataset.to_excel('train_dataset.xlsx')
train_dataset


,query,element,label,action,utterance
0,Email Address,"<input id=""fieldEmail"" name=""cm-ukutgu-ukutgu""...",email,NaN,NaN
1,Accept additional cookies,"<input id=""accept_all_cookies_button"" messaged...",cookie,NaN,NaN
2,Reject additional cookies,"<input id=""accept_essential_cookies_button"" me...",cookie,NaN,NaN
3,Your Name,"<input aria-describedby=""ph-a11y-email-job-nam...",first name,NaN,NaN
4,Recipient’s Email address,"<input aria-describedby="""" aria-label=""Please ...",email,NaN,NaN
...,...,...,...,...,...
2716,og in,"<button class=""careersite-button careersite-bu...",login,1.0,log in
2717,upload resume,"<a href=""javascript: void(0);"" title=""upload r...",cv,1.0,upload resume
2718,upload fileprivacy policy,"<a href=""https://vertriebskarriere.hansemerkur...",privacy,1.0,privacy policy
2719,back,"<a href=""https://bewerber-innendienst.hansemer...",cancel,1.0,back


In [61]:
test_dataset = pd.read_excel('./data/forms.xlsx')
c = test_dataset['label'].notna()
test_dataset = test_dataset.loc[c,:].reset_index(drop=True)
test_dataset

,query,element,label,sudo_label
0,Personal information\n CV or resume\n Browse C...,"<input accept=""application/pdf,.pdf"" name=""cv""...",cv,login
1,Name *\n,"<input name=""first_name"" placeholder=""First"" r...",first name,first name
2,This field contains invalid characters\n,"<input name=""last_name"" placeholder=""Last"" req...",last name,transcript
3,This field contains invalid characters\n Email...,"<input name=""email"" placeholder=""yourmail@doma...",email,email
4,Phone\n,"<input name=""phone"" placeholder=""+49 176 123 4...",phone number,phone number
...,...,...,...,...
95,cover letter\n upload your cover letter\n,"<input accept=""application/msword, application...",cover letter,NaN
96,github\n,"<input name=""custom_attribute_356304"" placehol...",github,NaN
97,ihre e-mail-adresse\n,"<button aria-label=""subscribe for newsletter"" ...",newsletter,NaN
98,(!)\n this field is required\n,"<button id=""uploaddropboxresume"" type=""submit""...",submit,NaN


In [36]:

person = {
    "first name": "majid",
    "last name": "sharghi foroushani",
    "pronoun": "he",
    "birth": "10/08/1997",
    "age": "27",
    "sex": "male",
    "skill": "NLP,LLM",
    "country": "Germany",
    "address": "Philip-9",
    "city": "Erlangen",
    "phone number": "+497773154",
    "available from": "6/24/2024",
    "salary": "10000",
    "German language level": "A1",
    "English language level": "C1",
    "visa": "I do not need visa.",
    "email": "sharghi.majid@gmail.com",
    "password": "123456",
    "find us": "Google",
    "xing": "www.xing.com",
    "job title": "I am a stduent.",
    "university": "FAU",
    "major": "AI",
    "privacy": "privacy",
    "search": "search",
    "submit": "submit",
    "cookie": "cookie",
    "cover letter": "cover letter",
    "linkedin": "www.linkedin.com",
    "github": "www.github.com",
    "login": "login",
    "register": "register",
    "cv": "/pah/to/cv.pdf",
    "twitter": "www.twitter.com",
    "years of experience": "4 years in deep learning",
    "username":"Majidsh97",
    "website language":"english",
    "transcript":"path/to/ts.pdf",
    "company":"company",
    "house number":"9",
    "italian language level":"A2",
    "ethnicity":"white",
    "picture":"path/to/p.jpg",
    "career level":"entry",
    "zip":"91052",
    "newsletter":"newsletter",
    "fake":"fake",
    "question":"question",
    "cancel":"cancel",
    "recommender":"my recommender",
    "website":"my website",
    'workherebefore':"No",
    "next":"next",
    "dropbox":"www.dropbox.com",
    "googledrive":"googledrive",
    "work hours":"8",
    "workduration":"6 months",
    "willing to relocate":"yes",
    "universityenrollment":"path to enrollment letter",
    "subscribe":"subscribe",
    "distance":"50 km",
    "captcha":"captcha",
    "forgot password":"no",
    "graduation year":"2024",
    "civic number":"54864",
    "person title":"Mr",
    "show password":"show password",
    
    
}
# {'years of experience', 'forgot password', 'next', 'username', 'website language', 'latitude', 'distance', 'transcript', 'english language level', 'cancel', 'company', 'longitude', 'house number', 'italian language level', 'title', 'captcha', 'civic number', 'zip', 'recommender', 'newsletter', 'show password', 'ethnicity', 'graduation year', 'job title', 'subscribe', 'picture', 'career level'}




In [ ]:
from transformers import Qwen2ForCausalLM
device = 'cuda'
model_name = 'Qwen/Qwen2-0.5B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(model_name,cache_dir=cache_dir)
model = Qwen2ForCausalLM.from_pretrained(model_name #Qwen/Qwen2-0.5B-Instruct
                                         #,attn_implementation="flash_attention_2"
                                         #,torch_dtype=torch.float16
                                         , device_map='auto',cache_dir=cache_dir).to('cuda')

model.eval()
#model.config['use_sliding_window']=True
print(model.config)

model

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Qwen2Config {
  "_name_or_path": "Qwen/Qwen2-0.5B-Instruct",
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "hidden_act": "silu",
  "hidden_size": 896,
  "initializer_range": 0.02,
  "intermediate_size": 4864,
  "max_position_embeddings": 32768,
  "max_window_layers": 24,
  "model_type": "qwen2",
  "num_attention_heads": 14,
  "num_hidden_layers": 24,
  "num_key_value_heads": 2,
  "rms_norm_eps": 1e-06,
  "rope_theta": 1000000.0,
  "sliding_window": 32768,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.41.2",
  "use_cache": true,
  "use_sliding_window": false,
  "vocab_size": 151936
}



Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
  )
  (lm_head): Linear(in_featur

In [ ]:
model.generation_config

GenerationConfig {
  "bos_token_id": 151643,
  "do_sample": true,
  "eos_token_id": [
    151645,
    151643
  ],
  "pad_token_id": 151643,
  "repetition_penalty": 1.1,
  "temperature": 0.7,
  "top_k": 20,
  "top_p": 0.8
}

In [ ]:
model = get_peft_model(model,LoraConfig(task_type=TaskType.CAUSAL_LM,target_modules=['q_proj','v_proj'
                                                                                     ,'k_proj'#,'o_proj'
                                                                                     ]))
model.print_trainable_parameters()
model
for n,p in model.named_parameters():
    if p.requires_grad==False:
        print(n)

trainable params: 737,280 || all params: 494,770,048 || trainable%: 0.1490
base_model.model.model.embed_tokens.weight
base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight
base_model.model.model.layers.0.self_attn.q_proj.base_layer.bias
base_model.model.model.layers.0.self_attn.k_proj.base_layer.weight
base_model.model.model.layers.0.self_attn.k_proj.base_layer.bias
base_model.model.model.layers.0.self_attn.v_proj.base_layer.weight
base_model.model.model.layers.0.self_attn.v_proj.base_layer.bias
base_model.model.model.layers.0.self_attn.o_proj.weight
base_model.model.model.layers.0.mlp.gate_proj.weight
base_model.model.model.layers.0.mlp.up_proj.weight
base_model.model.model.layers.0.mlp.down_proj.weight
base_model.model.model.layers.0.input_layernorm.weight
base_model.model.model.layers.0.post_attention_layernorm.weight
base_model.model.model.layers.1.self_attn.q_proj.base_layer.weight
base_model.model.model.layers.1.self_attn.q_proj.base_layer.bias
base_model.model.model

In [ ]:
#test_dataset.to_excel('forms.xlsx',index=0)

In [ ]:
s='['
for i in LABEL_INDEX_TO_KEY:
    s+=f'"{i}",'
    pass

s+=']'
s
#str(LABEL_INDEX_TO_KEY[0])

'["birth","major","cookie","search","city","English language level","years of experience","forgot password","next","username","address","website language","latitude","distance","cv","phone number","transcript","english language level","register","pronoun","university","cancel","job","company","longitude","age","house number","skill","italian language level","visa","title","twitter","captcha","civic number","zip","find us","recommender","available from","linkedin","submit","email","salary","newsletter","first name","German language level","show password","last name","ethnicity","github","cover letter","graduation year","sex","password","privacy","login","job title","subscribe","picture","country","xing","career level",]'

In [ ]:
"""def f(x):
    print(x)
    return '{"answer":"'+ x +'"}'

train_dataset['labels'] = train_dataset['labels'].map(f)
train_dataset"""


'def f(x):\n    print(x)\n    return \'{"answer":"\'+ x +\'"}\'\n\ntrain_dataset[\'labels\'] = train_dataset[\'labels\'].map(f)\ntrain_dataset'

In [43]:
import random


def get_random():
    d = {}
    for k in person.keys():
        d[k] = str(random.randint(1000, 10000))

    return d


# get_random()
def dict_to_str(p):
    s = "{"
    for k, v in p.items():
        s += f'"{k}":"{v}",'
        pass

    s += "}"
    return s


def get_one_chat_template(person_str, query_element_str, label_str=None):
    d = [
        {   #{"role":"system","content":""}
            "role": "user",
            #"content": """Your answer has to be in json format like {"answer":""}. Use this content in your answer:\n"""
            "content": """Your answer has to be in json format like {"answer":""}."""
            
            + person_str,
        },
        {
            "role": "user",
            #"content": "What is this HTML element about?\n" + query_element_str,
            "content": "What is this HTML element about?\n" + query_element_str,
            
            #"content": "Question:\n" + query_element_str,
            
        },
    ]
    if label_str:
        d.append(
            {"role": "assistant", "content": '{"answer":"' + label_str + '"}'},
        )
        
    return d


def get_one_msg(text, label):
    p = person #get_random()
    strp = ""#dict_to_str(p)
    rc = ""  # random.choice(list(p.keys()))+' '
    d = get_one_chat_template(strp, rc + text,label)#p[label])
    return d


def get_chat():

    d = []
    d+=train_dataset.apply( lambda x :get_one_msg(str(x["query"]) +'\n'+str(x['element']) , str(x["label"])),axis=1).tolist()
    d+=train_dataset_2.apply( lambda x :get_one_msg(str(x["query"])+'\n'+str(x['element']), str(x["label"])),axis=1).tolist()


    return d  # ,labels


get_chat()

[[{'role': 'user',
   'content': 'Your answer has to be in json format like {"answer":""}.'},
  {'role': 'user',
   'content': 'What is this HTML element about?\npersonal informationcv or resumebrowse cv to autocomplete your application. you can still fill your profile manually.\n<input accept="application/pdf,.pdf" name="cv" style="display: none;" tabindex="-1" type="file"/>'},
  {'role': 'assistant', 'content': '{"answer":"cv"}'}],
 [{'role': 'user',
   'content': 'Your answer has to be in json format like {"answer":""}.'},
  {'role': 'user',
   'content': 'What is this HTML element about?\nnan\n<input autocomplete="email" class="sc-bsgiji jskkmm" data-testid="textinput" id="email" name="email" required="" type="email" value=""/>'},
  {'role': 'assistant', 'content': '{"answer":"email"}'}],
 [{'role': 'user',
   'content': 'Your answer has to be in json format like {"answer":""}.'},
  {'role': 'user',
   'content': 'What is this HTML element about?\n*\n<input class="sc-kilemz pddck" 

In [44]:
def get_train_data():
    return tokenizer.apply_chat_template(get_chat(),add_generation_prompt=False ,truncation=True,max_length=512)  # ,return_tensors='pt')
    pass

tokenized_data = get_train_data()


no_label = tokenizer.apply_chat_template([  get_one_chat_template( ""#person_to_str(person)
                                                                  , str(x['query'])+x['element']) 
                                for i,x in test_dataset.iterrows()] ,truncation=True,max_length=512)  # ,return_tensors='pt')

print(len(tokenized_data))
print(tokenizer.decode(token_ids=tokenized_data[0]))
print(len(tokenized_data[0]))
print(tokenizer.decode(token_ids=no_label[0]))


NameError: name 'tokenizer' is not defined

In [ ]:
len(tokenized_data[0])

99

In [ ]:
tokenizer(['<|im_start|>assistant'])

{'input_ids': [[151644, 77091]], 'attention_mask': [[1, 1]]}

In [ ]:
mydc = DataCollatorForLanguageModeling(tokenizer,False)
def mmf(x):
    y = mydc(x)
    b = (y['labels']==77091).int().argmax()-1
    for i in range(0,b):
        y['labels'][0][i]=-100
        pass
    #print( y['labels'] )
    return y

mmf(no_label)['labels'].shape


torch.Size([100, 512])

In [ ]:
from transformers import TrainerCallback
"""
class ChangeDatasetCallback(TrainerCallback):
    def __init__(self):
        #super.__init__()
        pass

    def on_epoch_end(self, args, state, control, **kwargs):
        # Move to the next dataset at the end of each epoch
        #self.current_epoch += 1
        #new_dataset = self.datasets[self.current_epoch % len(self.datasets)]
        kwargs['train_dataloader'].dataset = get_train_data()
        #kwargs['eval_dataloader'].dataset = new_dataset['validation']
"""

class custom_dataset(Dataset):
    def __init__(self) -> None:
        super().__init__()
        self.dataset = get_train_data()
        self.len = len(self.dataset)
        pass

    def __getitem__(self, i) :
        if i==self.len-1:
            self.dataset = get_train_data()

        return self.dataset[i]
        pass

    def __len__(self):
        return self.len
        pass

trainer =  Trainer(model,
        train_dataset=custom_dataset(),
        data_collator=mmf,
        #compute_metrics=compute_metric,
        #callbacks=[ChangeDatasetCallback()],
        args=TrainingArguments(
          output_dir=output_dir,
          num_train_epochs=4,
          per_device_train_batch_size=1,
          per_device_eval_batch_size=1,
          logging_steps=300,

          #eval_accumulation_steps =2,


        ))

trainer.train()
model = model.eval()

[2024-07-04 06:22:18,645] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.1), only 1.0.0 is known to be compatible


Step,Training Loss
300,0.676900
600,0.368900
900,0.257300
1200,0.246100
1500,0.188600
1800,0.165400
2100,0.169300
2400,0.152100
2700,0.166800
3000,0.176000


/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: F

KeyboardInterrupt: 

In [ ]:
#trainer.model.save_pretrained(output_dir+'saved_84/')


/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
questions = [
"use html to create email"
,"""what is your gender?"""
,"""where are you living?(optional)"""
,"what is your salary expectation?*"
,"Which pronouns should we use to address you? (optional)"
,"how is your German?"
,"do you need visa spunsership?"
,"where are you working right now?"
,"what is your birth date?"
,"what are your hobbies?"
,"password"
,"what projects have you done?"
,"are you a text ai?"
,"what experice do you have in network engineering?"
,"what projects have you done in deep learning?"
,"how did you find us?"]

model.disable_adapter()

x= [get_one_chat_template(""#person_to_str(person)
                          ,q+"""<input type="text" style="" id="custom">""") for q in questions]
tokenized_questions = tokenizer.apply_chat_template(x)

with torch.no_grad():
    
    for i,x in enumerate(tokenized_questions):
            x = torch.tensor([x]).to('cuda')
            o = model.generate(input_ids=x,max_new_tokens=20,do_sample=False)
            print(questions[i])
            decoded = tokenizer.decode(o[0,x.shape[1]+3:])
            print(decoded)
            print('--------------------------------------------------------')
        
    

use html to create email
{"answer":"email"}<|im_end|>
--------------------------------------------------------
what is your gender?
{"answer":"find us"}<|im_end|>
--------------------------------------------------------
where are you living?(optional)
{"answer":"location"}<|im_end|>
--------------------------------------------------------
what is your salary expectation?*
{"answer":"salary"}<|im_end|>
--------------------------------------------------------
Which pronouns should we use to address you? (optional)
{"answer":"first name"}<|im_end|>
--------------------------------------------------------
how is your German?
{"answer":"German language"}<|im_end|>
--------------------------------------------------------
do you need visa spunsership?
{"answer":"visa"}<|im_end|>
--------------------------------------------------------
where are you working right now?
{"answer":"find us"}<|im_end|>
--------------------------------------------------------
what is your birth date?
{"answer":"bir

In [ ]:
model = model.to('cuda')
def get_answer(decoded):
    try:
                #print("ERROR: ",decoded)

                _s = decoded.find('{')
                _e = decoded.find('}',_s)+1
                #print(decoded[_s:_e])
                answer = json.loads(decoded[_s:_e])['answer']
                return answer
                
    except Exception as e:
                print("ERROR: ",e)
                print(decoded)
                print("---------------------------------")

                
def compute_metric():
    model.eval()
    acc=0
    with torch.no_grad():
        for i in range(len(no_label)):
            x = mydc([no_label[i]])['input_ids'].to('cuda')
            #print(x)
            o = model.generate(input_ids=x,max_new_tokens=20,do_sample=False)
            decoded = tokenizer.decode(o[0,x.shape[1]:])
            #print(decoded)
            #break

            #print(x[0],x[1])
            answer = get_answer(decoded)
            _label = test_dataset.loc[i,'label']
            #_label = person[test_dataset.loc[i,'label']]
            c = answer==_label

            if c:
                acc +=1
            else:
                print("NO_CORRECT:",answer,_label)
                print("---------------------------------")

                pass

    acc/=len(no_label)
    print("acc:",acc)



compute_metric()

/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:537: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


NO_CORRECT: pronoun sex
---------------------------------
ERROR:  Expecting value: line 1 column 1 (char 0)
="country_chile"> Chile</option><option value="country_china"> China</option><
---------------------------------
NO_CORRECT: None country
---------------------------------
NO_CORRECT: cv cover letter
---------------------------------
NO_CORRECT: pronoun sex
---------------------------------
NO_CORRECT: captcha privacy
---------------------------------
NO_CORRECT: privacy sex
---------------------------------
NO_CORRECT: search submit
---------------------------------
NO_CORRECT: find us cv
---------------------------------
NO_CORRECT: linkedin xing
---------------------------------
NO_CORRECT: job title country
---------------------------------
ERROR:  Expecting value: line 1 column 1 (char 0)
><option value="34">burgenland</option><option value="35">c
---------------------------------
NO_CORRECT: None country
---------------------------------
NO_CORRECT: pronoun sex
------------

In [ ]:
model.eval()
i =11
d = mydc(no_label[i:i+1])['input_ids'].to(device)
d = tokenizer.apply_chat_template( [#{"role":"system","content":""}
                             {"role":"user", "content":"""Your answer has to be in json format like {"answer":""}. Use this content in your answer:\n"""+s}
                            ,{"role":"user", "content":'email'}] , return_tensors='pt').to(device)
#d = tokenizer(['json {"answer":""} what is this html element about? <input type="email">'],return_tensors='pt')['input_ids'].to(device)
#print(d)
with torch.no_grad():

    o = model.generate(input_ids = d,max_length = 512,do_sample=True)
    print(tokenizer.decode(o[0]))



<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Your answer has to be in json format like {"answer":""}. Use this content in your answer:
["birth","major","cookie","search","city","English language level","years of experience","forgot password","next","username","address","website language","latitude","distance","cv","phone number","transcript","english language level","register","pronoun","university","cancel","job","company","longitude","age","house number","skill","italian language level","visa","title","twitter","captcha","civic number","zip","find us","recommender","available from","linkedin","submit","email","salary","newsletter","first name","German language level","show password","last name","ethnicity","github","cover letter","graduation year","sex","password","privacy","login","job title","subscribe","picture","country","xing","career level",]<|im_end|>
<|im_start|>user
email<|im_end|>
<|im_start|>assistant
{"answer":"johndoe@example.com"}<|im_end|>

In [ ]:
"""x =Qwen2ForCausalLM.from_pretrained('Qwen/Qwen2-0.5B-Instruct',cache_dir=cache_dir)
model = PeftModelForCausalLM.from_pretrained(x,output_dir+'saved/')"""